In [ ]:
import importlib

def install_package(package):
    try:
        importlib.import_module(package)
        print(f"{package} is already installed.")
    except ImportError:
        import sys
        import subprocess

        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"{package} has been installed.")

# Install pandas
install_package("pandas")

# Install openpyxl
install_package("openpyxl")

# Install itertools
install_package("itertools")

In [1]:
import pandas as pd

file_path = 'Arbitrage.xlsx'
df = pd.read_excel(file_path)

# Perform triangular arbitrage
def triangular_arbitrage():
    opportunities = []
    
    # Define and populate the conversion_rates dictionary
    conversion_rates = {}
    
    # Extract currency pair and price information from the DataFrame
    for index, row in df.iterrows():
        ticker_pair = row['Ticker Pair']
        price = row['Price']
        base_currency, quote_currency = ticker_pair.split('/')
        conversion_rates[(base_currency, quote_currency)] = price
    
    # Iterate over all possible triangular arbitrage opportunities
    for (base_currency, quote_currency) in conversion_rates.keys():
        for intermediate_currency in conversion_rates.keys():
            if intermediate_currency != (base_currency, quote_currency):
                if (quote_currency, intermediate_currency[1]) in conversion_rates and (intermediate_currency[1], base_currency) in conversion_rates:
                    cross_rate = (conversion_rates[(quote_currency, intermediate_currency[1])] / 
                                  conversion_rates[(intermediate_currency[1], base_currency)])
                    if cross_rate > conversion_rates[(base_currency, quote_currency)]:
                        opportunity = {
                            'Base Currency': base_currency,
                            'Quote Currency 1': quote_currency,
                            'Quote Currency 2': intermediate_currency[1],
                            'Rate 1': conversion_rates[(base_currency, quote_currency)],
                            'Rate 2': conversion_rates[(quote_currency, intermediate_currency[1])],
                            'Rate 3': conversion_rates[(intermediate_currency[1], base_currency)],
                            'Cross Rate': cross_rate
                        }
                        opportunities.append(opportunity)
    
    # Create a DataFrame from the opportunities list
    opportunities_df = pd.DataFrame(opportunities)
    
    # Convert the cross rate to the base currency
    for index, row in opportunities_df.iterrows():
        opportunities_df.at[index, 'Cross Rate'] = 1 / row['Cross Rate']
        
        # Calculate the profit percentage
        profit_percentage = ((100000000 * row['Rate 1'] * row['Rate 2'] * row['Rate 3']) / 100000000) 
        opportunities_df.at[index, 'Profit Percentage'] = profit_percentage
    
    opportunities_df = opportunities_df.sort_values('Profit Percentage', ascending=False)
    opportunities_df = opportunities_df.drop_duplicates(['Base Currency', 'Quote Currency 1', 'Quote Currency 2'])
    
    return opportunities_df

# Run the triangular_arbitrage function and print the opportunities DataFrame
arbitrage_opportunities_df = triangular_arbitrage()
print(arbitrage_opportunities_df)


        Base Currency Quote Currency 1 Quote Currency 2     Rate 1     Rate 2  \
2109687           KES              MAD              SEK   0.070000     1.0600   
2399232           MAD              SEK              KES   1.060000    13.8132   
390578            CNY              CHF              COP   0.120700  4705.3700   
399056            CNY              HKD              COP   1.094000   519.1300   
1639147           COP              CNY              HKD   0.001800     1.0940   
...               ...              ...              ...        ...        ...   
485200            SEK              PEN              VND   0.346200  6605.7300   
1857907           VND              SEK              UAH   0.000062     3.5728   
477369            SEK              UAH              VND   3.572800   639.8600   
1857086           VND              SEK              XAF   0.000062    57.0664   
487039            SEK              XAF              VND  57.066400    38.5198   

             Rate 3    Cros